In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # turn off warning

In [2]:
# read raw data with pandas
df = pd.read_csv("germlineMutationDataIARC TP53 Database, R19.txt",sep="\t")

In [3]:
# check the few ten rows
df.head(10)

,Family_ID,Family_code,Country,Population,Region,Development,Class,Generations_analyzed,Germline_mutation,MUT_ID,...,Age,Tumor_ID,Topography,Short_topo,Morphology,Age_at_diagnosis,Ref_ID,Other_infos,p53mut_ID,Add_Info
0,4,Bard94,USA,Northern America,Americas,More developed regions,LFL,2.0,TP53,3259,...,NaN,114.0,HEMATOPOIETIC AND RETICULOENDOTHELIAL SYSTEMS,HEMATOP. SYSTEM,"Leukemia, NOS",6.0,1,NaN,2,in ref 62
1,4,Bard94,USA,Northern America,Americas,More developed regions,LFL,2.0,TP53,3259,...,35.0,2.0,CERVIX UTERI,CERVIX UTERI,"Carcinoma, NOS",30.0,1,NaN,2,NaN
2,4,Bard94,USA,Northern America,Americas,More developed regions,LFL,2.0,TP53,3259,...,35.0,3.0,BRAIN,BRAIN,"Glioma, malignant (C71._)",35.0,1,NaN,2,NaN
3,4,Bard94,USA,Northern America,Americas,More developed regions,LFL,2.0,TP53,3259,...,NaN,4.0,ADRENAL GLAND,ADRENAL GLAND,Adrenal cortical carcinoma (C74.0),1.0,1,NaN,2,NaN
4,5,Bir16,UK,Northern Europe,Europe,More developed regions,LFS,2.0,TP53,2821,...,27.0,5.0,BREAST,BREAST,"Cancer, NOS",27.0,2,updated from ref 86,3,NaN
5,5,Bir16,UK,Northern Europe,Europe,More developed regions,LFS,2.0,TP53,2821,...,NaN,6.0,ADRENAL GLAND,ADRENAL GLAND,Adrenal cortical carcinoma (C74.0),1.0,2,updated from ref 86,3,NaN
6,5,Bir16,UK,Northern Europe,Europe,More developed regions,LFS,2.0,TP53,2821,...,NaN,7.0,"CONNECTIVE, SUBCUTANEOUS AND OTHER SOFT TISSUES",SOFT TISSUES,"Rhabdomyosarcoma, NOS",1.0,2,updated from ref 86,3,NaN
7,9,Bir-83,UK,Northern Europe,Europe,More developed regions,LFS,3.0,TP53,2143,...,13.0,23.0,"CONNECTIVE, SUBCUTANEOUS AND OTHER SOFT TISSUES",SOFT TISSUES,"Fibrosarcoma, NOS",3.0,2,updated from ref 86,4,NaN
8,9,Bir-83,UK,Northern Europe,Europe,More developed regions,LFS,3.0,TP53,2143,...,13.0,24.0,"BONES, JOINTS AND ARTICULAR CARTILAGE OF OTHER...",BONES (other),"Osteosarcoma, NOS (C40._, C41._)",10.0,2,updated from ref 86,4,NaN
9,9,Bir-83,UK,Northern Europe,Europe,More developed regions,LFS,3.0,TP53,2143,...,13.0,25.0,"BONES, JOINTS AND ARTICULAR CARTILAGE OF OTHER...",BONES (other),"Osteosarcoma, NOS (C40._, C41._)",13.0,2,updated from ref 86,4,NaN


In [4]:
# remove rows without age of tumor onset or individual code
df_rm_missing = df.dropna(subset=["Individual_code","Age_at_diagnosis"])

print("#rows before: {}".format(df.shape[0]))
print("#rows after: {}".format(df_rm_missing.shape[0]))
print("{:.1f}% of rows without Age_at_diagnosis or Individual_code".format(100-df_rm_missing.shape[0]/df.shape[0]*100))

#rows before: 3842
#rows after: 2954
23.1% of rows without Age_at_diagnosis or Individual_code


In [5]:
# combine the tumor types of first tumor onset for each patient

# get first age of onset for each patient
d_ID_minAge = dict(df_rm_missing.groupby('Individual_code')['Age_at_diagnosis'].agg('min'))
l_IDminAge = ["".join([k,str(v)]) for k,v in d_ID_minAge.items()]
    

# concatenate 'Individual_code' and 'Age_at_diagnosis' as selecting index for first age of onset
col = df_rm_missing.columns
df_rm_missing.loc[:,'ID_age'] = df_rm_missing[['Individual_code','Age_at_diagnosis']].apply(lambda x: "".join(map(str,list(x))),axis=1)


# remove rows that are not first of onset 
df_minAge = df_rm_missing[df_rm_missing['ID_age'].isin(l_IDminAge)][col]

# combine tumor types
gb_ID_combined = dict(df_minAge.groupby('Individual_code')['Short_topo'].apply(lambda x: ";".join(sorted(list(set(x))))))
gb_ID_count = dict(df_minAge.groupby('Individual_code')['Short_topo'].apply(lambda x: len(list(set(x)))))
df_minAge['Short_topo_combined'] = df_minAge.apply(lambda row: gb_ID_combined[row['Individual_code']],axis=1)
df_minAge['Short_topo_combined_count'] = df_minAge.apply(lambda row: gb_ID_count[row['Individual_code']],axis=1)



In [6]:
# Remove duplicate id
print('#rows before removing duplicated ID: {}'.format(df_minAge.shape[0]))
df_minAge = df_minAge.drop_duplicates(subset='Individual_code')
print('#rows after removing duplicated ID: {}'.format(df_minAge.shape[0]))

#rows before removing duplicated ID: 2318
#rows after removing duplicated ID: 2192


In [7]:
# Select column to analyze
cols = open("column_to_analyze.txt").readlines()[0].strip().split("\t")
df_minAge[cols].to_csv("LFS_clean.tsv",sep="\t",index=False)